<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3: Web APIs & NLP

## Contents
- [Exploratory Data Analysis](#Part-3:-Exploratory-Data-Analysis)
- [Feature Engineering](#Feature-Engineering)
- [Data Dictionary](#Data-Dictionary)

--- 
# Part 5 : Modeling & Results

--- 